<a href="https://colab.research.google.com/github/napoles-uach/DGL_Smell/blob/main/dgl_smell_fruity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [24]:
pip install dgl

In [25]:
pip install ogb

In [26]:
pip install rdkit-pypi

In [27]:
!wget https://raw.githubusercontent.com/napoles-uach/DGL_Smell/main/train.csv

--2021-08-07 01:16:49--  https://raw.githubusercontent.com/napoles-uach/DGL_Smell/main/train.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 197190 (193K) [text/plain]
Saving to: ‘train.csv.1’

train.csv.1         100%[===================>] 192.57K  --.-KB/s    in 0.01s   

2021-08-07 01:16:49 (16.5 MB/s) - ‘train.csv.1’ saved [197190/197190]



In [28]:
import pandas as pd
df=pd.read_csv('train.csv')
df.head()

,SMILES,SENTENCE
0,C/C=C/C(=O)C1CCC(C=C1C)(C)C,"fruity,rose"
1,COC(=O)OC,"fresh,ethereal,fruity"
2,Cc1cc2c([nH]1)cccc2,"resinous,animalic"
3,C1CCCCCCCC(=O)CCCCCCC1,"powdery,musk,animalic"
4,CC(CC(=O)OC1CC2C(C1(C)CC2)(C)C)C,"coniferous,camphor,fruity"


In [29]:
import dgl
from dgl.data import DGLDataset
import torch
import torch as th
import os
from ogb.utils.features import (allowable_features, atom_to_feature_vector,
 bond_to_feature_vector, atom_feature_vector_to_dict, bond_feature_vector_to_dict) 
from rdkit import Chem
import numpy as np

In [30]:
def smiles2graph(smiles_string):
    """
    Converts SMILES string to graph Data object
    :input: SMILES string (str)
    :return: graph object
    """

    mol = Chem.MolFromSmiles(smiles_string)

    A = Chem.GetAdjacencyMatrix(mol)
    A = np.asmatrix(A)
    nnodes=len(A)
    nz = np.nonzero(A)
    edge_list=[]
    src=[]
    dst=[]

    for i in range(nz[0].shape[0]):
      src.append(nz[0][i])
      dst.append(nz[1][i])

    u, v = src, dst
    g = dgl.graph((u, v))
    bg=dgl.to_bidirected(g)

    return bg

In [31]:
def feat_vec(smiles_string):
    """
    Returns atom features for a molecule given a smiles string
    """
    # atoms
    mol = Chem.MolFromSmiles(smiles_string)
    atom_features_list = []
    for atom in mol.GetAtoms():
        atom_features_list.append(atom_to_feature_vector(atom))
    x = np.array(atom_features_list, dtype = np.int64)
    return x

In [32]:
lista_senten=df['SENTENCE'].to_list()
olores=[]
labels=[]
i=0
for olor in lista_senten:
  olor=olor.split(",")
  if 'fruity' in olor:
    labels.append(1)
    olores.append(olor)
  else:
    labels.append(0)
  i+=1
  #olores.append(olor)
# da formato correcto a las sentencias en forma de lista
#olores

In [33]:
# This block makes a list of graphs
lista_mols=df['SMILES'].to_list()

j=0
graphs=[]
execptions=[]
for mol in lista_mols:
  
  g_mol=smiles2graph(mol)

  try:
    g_mol.ndata['feat']=torch.tensor(feat_vec(mol)) 
  except:
    execptions.append(j)
   

  graphs.append(g_mol) #agrega grafo de mol
  j+=1

In [34]:
# Some smiles are not well processed, so they are droped
ii=0
for i in execptions:
  graphs.pop(i-ii)
  labels.pop(i-ii)
  ii+=1



In [35]:
i=0
for grap in graphs:

  try:
    grap.ndata['feat']
  except:
    print(i)
  i+=1
  #if grap.ndata['feat'] is None:
  #  print("ok")

In [36]:
execptions

[70,
 265,
 305,
 454,
 1542,
 1574,
 1915,
 1995,
 2087,
 2149,
 2389,
 2540,
 2587,
 2899,
 3347,
 3671,
 4008,
 4014,
 4250]

In [37]:
import dgl
import torch
import torch.nn as nn
import torch.nn.functional as F

In [38]:
labels=F.one_hot(torch.LongTensor(labels), num_classes=2)

In [39]:
labels

tensor([[0, 1],
        [0, 1],
        [1, 0],
        ...,
        [1, 0],
        [1, 0],
        [1, 0]])

In [40]:
class SyntheticDataset(DGLDataset):
    def __init__(self):
        super().__init__(name='synthetic')

    def process(self):
        #edges = pd.read_csv('./graph_edges.csv')
        #properties = pd.read_csv('./graph_properties.csv')
        self.graphs = graphs
        self.labels = labels#torch.LongTensor(labels)

    def __getitem__(self, i):
        return self.graphs[i], self.labels[i]

    def __len__(self):
        return len(self.graphs)

dataset = SyntheticDataset()
#graph, label = dataset[0]
#print(graph, label)

In [41]:
from dgl.dataloading import GraphDataLoader
from torch.utils.data.sampler import SubsetRandomSampler

num_examples = len(dataset)
num_train = int(num_examples * 0.8)

train_sampler = SubsetRandomSampler(torch.arange(num_train))
test_sampler = SubsetRandomSampler(torch.arange(num_train, num_examples))

train_dataloader = GraphDataLoader(
    dataset, sampler=train_sampler, batch_size=5, drop_last=False)
test_dataloader = GraphDataLoader(
    dataset, sampler=test_sampler, batch_size=5, drop_last=False)

In [42]:
for batched_graph, labels in train_dataloader:
    print(batched_graph,labels)


Graph(num_nodes=47, num_edges=92,
      ndata_schemes={'feat': Scheme(shape=(9,), dtype=torch.int64)}
      edata_schemes={}) tensor([[0, 1],
        [1, 0],
        [1, 0],
        [1, 0],
        [1, 0]])
Graph(num_nodes=65, num_edges=136,
      ndata_schemes={'feat': Scheme(shape=(9,), dtype=torch.int64)}
      edata_schemes={}) tensor([[1, 0],
        [1, 0],
        [1, 0],
        [0, 1],
        [1, 0]])
Graph(num_nodes=64, num_edges=126,
      ndata_schemes={'feat': Scheme(shape=(9,), dtype=torch.int64)}
      edata_schemes={}) tensor([[0, 1],
        [1, 0],
        [1, 0],
        [0, 1],
        [0, 1]])
Graph(num_nodes=65, num_edges=122,
      ndata_schemes={'feat': Scheme(shape=(9,), dtype=torch.int64)}
      edata_schemes={}) tensor([[1, 0],
        [1, 0],
        [0, 1],
        [1, 0],
        [1, 0]])
Graph(num_nodes=51, num_edges=100,
      ndata_schemes={'feat': Scheme(shape=(9,), dtype=torch.int64)}
      edata_schemes={}) tensor([[1, 0],
        [1, 0],
        [1

In [20]:
import dgl
import torch
import torch.nn as nn
import torch.nn.functional as F

In [22]:

import numpy as np
#g.add_edges(g.nodes(), g.nodes())
optimizer = th.optim.Adam(net.parameters(), lr=1e-2)

#optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

for epoch in range(20):
    #for batched_graph, labels in train_dataloader:
    net.train()
    pred = net(graphs[10], graphs[10].ndata['feat'].float())
        #loss = F.cross_entropy(pred, labels)
        #optimizer.zero_grad()
        #loss.backward()
        #optimizer.step()

#loss_list=[]
#acc_list=[]
#all_logits=[]
#for epoch in range(200):
#    net.train()
#    logits = net(graphs[0], graphs[0].ndata['feat'])
#    pred=sum(logits)
#    logp = F.log_softmax(pred, 0)



    #pred = model(batched_graph, batched_graph.ndata['attr'].float())
    #loss = F.cross_entropy(pred, labels[0])
    
    #print(logits)
    #logp = F.log_softmax(logits, 1)
    #all_logits.append(logp)
#    loss = F.nll_loss(logp[train_mask], labels[train_mask])

#    optimizer.zero_grad()
#    loss.backward()
#    optimizer.step()

#    acc = evaluate(net, g, features, labels, test_mask)
#    loss_list.append(loss.item())
#    acc_list.append(acc)

In [46]:
from dgl.nn import GraphConv

class GCN(nn.Module):
    def __init__(self, in_feats, h_feats, num_classes):
        super(GCN, self).__init__()
        self.conv1 = GraphConv(9, 16)
        self.conv2 = GraphConv(16, 2)

    def forward(self, g, in_feat):
        h = self.conv1(g, in_feat)
        h = F.relu(h)
        h = self.conv2(g, h)
        g.ndata['h'] = h
        return dgl.mean_nodes(g, 'h')

In [50]:
# Create the model with given dimensions
model = GCN(9, 16, 2)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

for epoch in range(20):
    for batched_graph, labels in train_dataloader:
        pred = model(batched_graph, batched_graph.ndata['feat'].float())
        #print(pred,labels)
        loss = F.cross_entropy(pred, labels)
#        optimizer.zero_grad()
#        loss.backward()
#        optimizer.step()
#
#num_correct = 0
#num_tests = 0
#for batched_graph, labels in test_dataloader:
#    pred = model(batched_graph, batched_graph.ndata['attr'].float())
#    num_correct += (pred.argmax(1) == labels).sum().item()
#    num_tests += len(labels)
#
#print('Test accuracy:', num_correct / num_tests)

RuntimeError: ignored